In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

In [ ]:
from datasets import load_dataset, Audio
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
import torchaudio

# Load your custom dataset
dataset = load_dataset("path_to_your_dataset")

# Load processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base", 
                                       vocab_size=processor.tokenizer.vocab_size)

# Preprocessing function
def preprocess(batch):
    # Load audio
    audio = torchaudio.load(batch["audio_path"])[0]
    batch["input_values"] = processor(audio.numpy(), sampling_rate=16000, return_tensors="pt").input_values[0]
    batch["labels"] = processor.tokenizer(batch["transcription"], return_tensors="pt").input_ids[0]
    return batch

# Apply preprocessing
processed_dataset = dataset.map(preprocess)



In [ ]:
from transformers import Trainer, TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="./wav2vec2-dali",
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    logging_steps=100,
    save_steps=500,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=processor
)

# Train the model
trainer.train()

In [ ]:
model.save_pretrained("./wav2vec2-dali")
processor.save_pretrained("./wav2vec2-dali")